# Setup

Import Semantic Kernel SDK from pypi.org

In [1]:
# Note: if using a virtual environment, do not run this cell
%pip install -U semantic-kernel
from semantic_kernel import __version__

__version__

Note: you may need to restart the kernel to use updated packages.


'1.24.0'

# Agent

In [26]:
import asyncio
from typing import Annotated

from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistory, FunctionCallContent, FunctionResultContent
from semantic_kernel.functions import KernelArguments, kernel_function

In [42]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id))
    return kernel

In [41]:
import yaml

def create_chat_completion_agent(kernel: Kernel, definition_file_path: str, plugins: list) -> ChatCompletionAgent:
    with open(definition_file_path, 'r') as file:
        definition = yaml.safe_load(file)

    return ChatCompletionAgent(
        kernel=kernel,
        name=definition['name'],
        plugins = plugins,
        description=definition['description'],
        instructions=definition['instructions']
    )

## SQL Agent

In [ ]:
import sqlite3
import pandas as pd

# Sample data for the climbing product catalog
data = {
    "Product ID": ["P001", "P002", "P003", "P004", "P005"],
    "Product Name": ["Climbing Rope", "Climbing Shoes", "Carabiner", "Harness", "Chalk Bag"],
    "Category": ["Ropes", "Shoes", "Hardware", "Harnesses", "Accessories"],
    "Price": [100, 150, 20, 75, 15],
    "Stock": [50, 30, 100, 40, 200],
    "Description": [
        "Durable and strong climbing rope suitable for all terrains.",
        "High-performance climbing shoes for advanced climbers.",
        "Lightweight and secure carabiner for all climbing needs.",
        "Comfortable and adjustable harness for safety.",
        "Compact and durable chalk bag for better grip."
    ]
}

# Converting the dictionary to a DataFrame
df = pd.DataFrame(data)

# Connecting to a SQLite database (it will create a new one if it doesn't exist)
conn = sqlite3.connect('climbing_product_catalog.db')

# Storing the DataFrame in the SQLite database
df.to_sql('product_catalog', conn, index=False, if_exists='replace')

# Committing and closing the connection
conn.commit()
conn.close()

In [108]:
from Plugins.queryDb import QueryDbPlugin

sql_agent = create_chat_completion_agent(
    kernel = _create_kernel_with_chat_completion('sql_agent'),
    definition_file_path = "Prompts/sql.yaml",
    plugins=[QueryDbPlugin('climbing_product_catalog.db')]
)

In [109]:
# Define the chat history
from semantic_kernel.contents import ChatHistory
chat = ChatHistory()

# Add the user message
chat.add_user_message("how many products are in stock?")


# Generate the agent response
response = await sql_agent.get_response(chat)
response.content

'There are 420 products currently in stock.'

## Concierge Agent

In [67]:
concierge_agent = create_chat_completion_agent(
    kernel = _create_kernel_with_chat_completion('concierge_agent'),
    definition_file_path = "Prompts/concierge.yaml",
    plugins=[]
)

## Cart Agent

In [65]:
from Plugins.cart_plugin import CartPlugin


cart_agent = create_chat_completion_agent(
    kernel = _create_kernel_with_chat_completion('cart_agent'),
    definition_file_path = "Prompts/cart.yaml",
    plugins=[CartPlugin('http://localhost:3000')]
)

In [66]:
# Define the chat history
from semantic_kernel.contents import ChatHistory
chat = ChatHistory()

# Add the user message
chat.add_user_message("add climbing shoes to my cart, cost is 100$")


# Generate the agent response
response = await concierge_agent.get_response(chat)
response.content

'Climbing shoes have been added to your cart for $100. Is there anything else you would like to add to your cart or are you ready to check out?'

# Group chat

In [121]:
import asyncio
import os

from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatHistoryTruncationReducer
from semantic_kernel.functions import KernelFunctionFromPrompt

history_reducer = ChatHistoryTruncationReducer(target_count=1)

In [97]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies import TerminationStrategy
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "approved" in history[-1].content.lower()

In [120]:
selection_function = KernelFunctionFromPrompt(
    function_name="selection", 
    prompt=f"""
Examine the provided RESPONSE and choose the next participant.
State only the name of the chosen participant without explanation.


Choose only from these participants:
- {concierge_agent}
- {sql_agent}
- {cart_agent}

Rules:
- {concierge_agent} is the one greeting the user and invoking the other agents when needed.
- {sql_agent} is the one that can query the database and return the results whenever user asks about available items.
- {cart_agent} is the one that can add items to the cart.
- {concierge_agent} will always make sure whether the user is happy with the result or not.


RESPONSE:
{{{{$lastmessage}}}}
"""
)

In [112]:
termination_keyword = "yes"

termination_function = KernelFunctionFromPrompt(
    function_name="termination", 
    prompt=f"""
Examine the RESPONSE and determine whether the content has been deemed satisfactory.
If the content is satisfactory, respond with a single word without explanation: {termination_keyword}.
If specific suggestions are being provided, it is not satisfactory.
If no correction is suggested, it is satisfactory.

RESPONSE:
{{{{$lastmessage}}}}
"""
)

In [122]:
agents=[concierge_agent, sql_agent, cart_agent]

chat = AgentGroupChat(
    agents=agents,
    selection_strategy=KernelFunctionSelectionStrategy(
        initial_agent=concierge_agent,
        function=selection_function,
        kernel=_create_kernel_with_chat_completion('selection_function'),
        result_parser=lambda result: str(result.value[0]).strip() if result.value[0] is not None else WRITER_NAME,
        history_variable_name="lastmessage",
        history_reducer=history_reducer,
    ),
    termination_strategy=KernelFunctionTerminationStrategy(
        agents=[concierge_agent],
        function=termination_function,
        kernel=_create_kernel_with_chat_completion('termination_function'),
        result_parser=lambda result: termination_keyword in str(result.value[0]).lower(),
        history_variable_name="lastmessage",
        maximum_iterations=10,
        history_reducer=history_reducer,
    ),
)

In [123]:
import asyncio
is_complete = False
async def main():
    while not is_complete:
        # Get user input
        user_input = input("You: ")

        # Exit the loop if the user types 'exit'
        if user_input.lower() == 'exit':
            print("Exiting chat...")
            break
        if user_input.lower() == "reset":
            await chat.reset()
            print("[Conversation has been reset]")
            continue

        # Add the user input as a chat message
        await chat.add_chat_message(message=user_input)
        print(f"# User: {user_input}")

        try:
            # Invoke the chat and print the responses
            async for content in chat.invoke():
                print(f"# {content.name}: {content.content}")
        except KeyError as e:
            if "Symbols.VAR_PREFIX: input" in str(e):
                print("Error: Required input variable not found in the KernelArguments.")
            else:
                raise e
        chat.is_complete = False

# Run the main function
await main()

# User: ji
# concierge_agent: Hello! Thanks for reaching out. How can I assist you with your climbing gear needs today? Are you looking for any specific type of climbing gear or have any particular preferences in mind?
# User: list me the products
# sql_agent: Here is a list of products available in our catalog:

1. **Climbing Rope**
   - Product ID: P001
   - Category: Ropes
   - Price: $100.00
   - Stock: 50
   - Description: Durable and strong climbing rope suitable for all terrains.

2. **Climbing Shoes**
   - Product ID: P002
   - Category: Shoes
   - Price: $150.00
   - Stock: 30
   - Description: High-performance climbing shoes for advanced climbers.

3. **Carabiner**
   - Product ID: P003
   - Category: Hardware
   - Price: $20.00
   - Stock: 100
   - Description: Lightweight and secure carabiner for all climbing needs.

4. **Harness**
   - Product ID: P004
   - Category: Harnesses
   - Price: $75.00
   - Stock: 40
   - Description: Comfortable and adjustable harness for safety